<a href="https://colab.research.google.com/github/jhklee/SDC_new_issues_CRSP/blob/main/SDC_new_issues_CRSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math
from tqdm import tqdm
import ast
from typing import Hashable, List
from datetime import datetime
from dateutil.parser import parse

# SDC book manager list generate

usually the SDC platinum gives large data sets through txt files

In [ ]:
sdc_df = pd.read_fwf('/content/drive/MyDrive/UT_research/VC_IB/data/20070101_20230922.txt')
sdc_df

In [ ]:
# fill in nation 2 columns arbitraryly
sdc_df['Unnamed: 6'][0] = 'nation2'
sdc_df['Unnamed: 5'] = sdc_df['Unnamed: 5']  + sdc_df['Unnamed: 6']
sdc_df

In [ ]:
sdc_df_content = sdc_df.iloc[6:,:]
sdc_df_content

In [ ]:
sdc_df_columns = sdc_df.iloc[:6,:]
sdc_df_columns

In [ ]:
# combining the column names that are vertically writtin in excel.
new_row=[]
for i in range (sdc_df_columns.shape[1]):
  nalist = [text.lower() for text in sdc_df_columns.iloc[:,i] if pd.isnull(text) == False]
  nalist_1 = "_".join(nalist)
  new_row.append(nalist_1)

# preproccess column name
new_row_1 = [text.replace(' ','_').replace('-','_').replace('__','_').replace('___','_').replace('__','_') for text in new_row]
new_row_1

In [ ]:
sdc_df_content.columns = new_row_1
sdc_df_content

In [ ]:
sdc_ipo_df = sdc_df_content.reset_index(drop =True)
sdc_ipo_df

In [ ]:
sdc_ipo_df_2 =  sdc_ipo_df[sdc_ipo_df['ipo_flag_(y/n)'].isnull()]
sdc_ipo_df_2

# Combining the book writers that are vertically writtin in excel.

In [ ]:
# select the usefull columns
selected_list_2 = ['jkl_index', 'gross_spread_($)', 'manage_ment_fee_($)', 'under_writing_fee_($)',
                   'selling_conces_sion_($)','re_allow_ance_fee_($)','gross_spread_in_this_mkt_($_mil)',
                   'principal_amount_($_mil)','proceeds_amt_in_this_mkt_($_mil)',
                   'offer_price','type_of_security','description', 'marketplace',
                   'primary_exchange_where_issue_will_be_listed','original_high_filing_price',
                   'original_low_filing_price','original_middle_of_filing_price_range',
                   'shares_filed_in_this_mkt','amt_filed_in_this_mkt_($_mil)', 'reit_type_code','depos_itary',
                   'unit_inv._trust','deal_number', 'closed_end_fund/trust_flag_(y/n)', 'cusip', '9_digit_cusip',
                   'gross_spread_as_%_of_prncpl_amt_in_this_mkt','low_price_of_filing_price_rnge', 'high_price_of_filing_price_rnge',
                   'change_in_value_of_shares_this_market_between_offer_and_1_day_after_offer_(us$_mil)',
                   'market_value/_market_capitalization', 'market_value_after_offer_from_prospectus_($_mil)',
                   'market_value_pro_forma_offer_($_mil)', 'subscription_level','over_subscription_flag', 'subscription_shares']

In [ ]:
# use selected columns
selected_list = ['jkl_index','deal_number','filing_date', 'issue_date','issuer', 'main_sic_code', 'nationnation2','bookrunner(s)',
                 'all_managers','all_mgr_role_code','bookrunner(s).2','cusip','9_digit_cusip']
sdc_ipo_0 = sdc_ipo.copy().loc[:,selected_list]
sdc_ipo_0

In [ ]:
# we drop null rows
drop_list = ['deal_number','filing_date', 'issue_date','issuer', 'main_sic_code', 'nationnation2','bookrunner(s)',
                 'all_managers','bookrunner(s).2','all_mgr_role_code','cusip','9_digit_cusip']
sdc_ipo_1 = sdc_ipo_0.dropna(subset=drop_list, how='all')

# rename columns
sdc_ipo_1 = sdc_ipo_1.rename(columns = {'nationnation2':"nation", "bookrunner(s)": "lead_underwriter_code","bookrunner(s).2": "lead_underwriter"})
sdc_ipo_1

In [ ]:
# front fill make it easy to groupby
sdc_ipo_1['deal_number'] = sdc_ipo_1['deal_number'].ffill()

In [ ]:
# groupby by deal number
deal_num =  sdc_ipo_1['deal_number'].unique()
deal_num

array([8.39870009e+08, 4.52914002e+08, 4.44583002e+08, ...,
       4.13486300e+09, 4.13337200e+09, 4.10478300e+09])

In [ ]:
# check the rows without any manager role code
sdc_ipo_1[sdc_ipo_1['all_mgr_role_code'].isnull()]
sdc_ipo_1[sdc_ipo_1['jkl_index'] == 22042]

,jkl_index,deal_number,filing_date,issue_date,issuer,main_sic_code,nation,lead_underwriter_code,all_managers,all_mgr_role_code,lead_underwriter,cusip,9_digit_cusip
22042,22042,2.280765e+09,NaN,NaN,NaN,NaN,NaN,/SBI-CA(JB),Smith Barney Canada Inc,JB,Smith Barney Canada Inc,NaN,NaN


In [ ]:
# make columns
sdc_ipo_1['lead_underwriter_list'] = ''
sdc_ipo_1['JB_JL_jointlead'] = ''
sdc_ipo_1['CM_CL_comanager'] = ''

SDC has six codes for manager roles: "BM" (book manager), "JB" (joint book), "JL" (joint lead), "CM" (co-manager), "SD" (syndicate member) and "GL"(general lead). Managers that are coded as "GL" are usually also coded as either "BM" or "JB". Therefore, to avoid double counting we ignore "GL" code. Codes "BM", "JB" and "JL" all correspond to the lead underwriters. Code "CM" indicates a co-manager role and "SD" indicates other syndicate members.

In [ ]:
temp_df_list = []

for deal in tqdm(deal_num):
  # bring df by each deal
  temp_df_1 = sdc_ipo_1[sdc_ipo_1['deal_number']== deal]
  # list up the book runners
  temp2 = temp_df_1[(temp_df_1['all_mgr_role_code'] =="BM")|(temp_df_1['all_mgr_role_code'] =="JB")|(temp_df_1['all_mgr_role_code'] =="JL")].copy()['all_managers']
  temp3 = temp_df_1[(temp_df_1['all_mgr_role_code'] =="JB")|(temp_df_1['all_mgr_role_code'] =="JL")].copy()['all_managers']
  temp4 = temp_df_1[(temp_df_1['all_mgr_role_code'] =="CM")|(temp_df_1['all_mgr_role_code'] =="CL")].copy()['all_managers']
  # put the data inside the first row sell
  temp_df_1['lead_underwriter_list'].iloc[0] = list(temp2)
  temp_df_1['JB_JL_jointlead'].iloc[0] = list(temp3)
  temp_df_1['CM_CL_comanager'].iloc[0] = list(temp4)

  #save the first row in the list for concate
  temp_df_list.append(pd.DataFrame(temp_df_1.iloc[0,:]).T)

  0%|          | 0/33779 [00:00<?, ?it/s]<ipython-input-12-385f95990c93>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df_1['lead_underwriter_list'].iloc[0] = list(temp2)
<ipython-input-12-385f95990c93>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df_1['JB_JL_jointlead'].iloc[0] = list(temp3)
<ipython-input-12-385f95990c93>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df_1['CM_CL_comanager'].iloc[0] = list(

In [ ]:
sdc_ipo_2 = pd.concat(temp_df_list)
sdc_ipo_2

,jkl_index,deal_number,filing_date,issue_date,issuer,main_sic_code,nation,lead_underwriter_code,all_managers,all_mgr_role_code,lead_underwriter,cusip,9_digit_cusip,lead_underwriter_list,JB_JL_jointlead,CM_CL_comanager
0,0,839870009.0,10/07/94,01/05/95,Genetronics Biomedical Corp,2836,UnitedStates,CANACCORD,Canaccord Capital Corp,BM,Canaccord Capital Corp,371916,371916107,[Canaccord Capital Corp],[],[]
5,5,452914002.0,09/19/94,01/10/95,DiaSys Corp,3841,UnitedStates,TASIN,Tasin & Company,BM,"Tasin & Company, Inc.",252838,252838107,[Tasin & Company],[],[]
9,9,444583002.0,10/20/94,01/10/95,Hauppauge Digital Inc,3577,UnitedStates,LIEBERBAUM,Lew Lieberbaum & Co.,BM,"Lew Lieberbaum & Co., Inc.",419131,419131206,[Lew Lieberbaum & Co.],[],[]
11,11,444417002.0,11/08/94,01/11/95,MRV Communications Inc,3674,UnitedStates,HAMPSHIRE,Hampshire Securities Corp,BM,Hampshire Securities Corp,553477,553477100,[Hampshire Securities Corp],[],[]
16,16,444885002.0,05/10/94,01/12/95,Developers Diversified Realty,6798,UnitedStates,DWR,Dean Witter Reynolds Inc,BM,Dean Witter Reynolds Inc,251591,251591103,[Dean Witter Reynolds Inc],[],"[Alex Brown & Sons Inc, AG Edwards & Sons Inc,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226352,226352,4124156002.0,11/17/20,09/19/23,Klaviyo Inc,7372,UnitedStates,GS/MS-LLC(JB),Goldman Sachs & Co,JB,Goldman Sachs & Co,49845K,49845K101,"[Goldman Sachs & Co, Morgan Stanley & Co LLC, ...","[Goldman Sachs & Co, Morgan Stanley & Co LLC, ...","[Barclays Capital Inc, Mizuho Securities USA L..."
226364,226364,4051569002.0,03/13/23,09/19/23,SR Bancorp Inc,6021,UnitedStates,KEEFE,Keefe Bruyette & Woods Inc,BM,Keefe Bruyette & Woods Inc,5N5634,NaN,[Keefe Bruyette & Woods Inc],[],[]
226366,226366,4134863002.0,09/19/23,09/19/23,Permian Resources Corp,1311,UnitedStates,JPM-SEC-LLC,JP Morgan Securities LLC,JB,JP Morgan Securities LLC,71424F,71424F105,"[JP Morgan Securities LLC, BofA Securities Inc...","[JP Morgan Securities LLC, BofA Securities Inc...","[PNC Capital Markets LLC, BTIG LLC, Fifth Thir..."
226382,226382,4133372002.0,09/18/23,09/20/23,Madison Sq Garden Ent Corp,7999,UnitedStates,BOFA-SEC-INC/G,BofA Securities Inc,JB,BofA Securities Inc,558256,558256103,"[BofA Securities Inc, Goldman Sachs & Co, JP M...","[BofA Securities Inc, Goldman Sachs & Co, JP M...",[]


In [ ]:
sdc_ipo_fin_variable = sdc_ipo.loc[:,selected_list_2]
sdc_ipo_fin_variable

,jkl_index,gross_spread_($),manage_ment_fee_($),under_writing_fee_($),selling_conces_sion_($),re_allow_ance_fee_($),gross_spread_in_this_mkt_($_mil),principal_amount_($_mil),proceeds_amt_in_this_mkt_($_mil),offer_price,...,gross_spread_as_%_of_prncpl_amt_in_this_mkt,low_price_of_filing_price_rnge,high_price_of_filing_price_rnge,change_in_value_of_shares_this_market_between_offer_and_1_day_after_offer_(us$_mil),market_value/_market_capitalization,market_value_after_offer_from_prospectus_($_mil),market_value_pro_forma_offer_($_mil),subscription_level,over_subscription_flag,subscription_shares
0,0,0.067,Comb.,Comb.,NaN,NaN,0.167,2.231,2.2,1.25,...,7.504,NaN,NaN,NaN,"103,640",NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226382,226382,1.300,0.260,0.260,0.78,NaN,10.688,232.375,232.4,32.500,...,4.000,NaN,NaN,2.503,"1,451,450","1,387.9","1,387.9",NaN,NaN,NaN
226383,226383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226384,226384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226385,226385,0.385,Comb.,Comb.,NaN,NaN,0.385,5.500,5.5,5.500,...,7.000,NaN,NaN,NaN,NaN,281.0,281.0,NaN,NaN,NaN


In [ ]:
sdc_ipo_merge_1 =  pd.merge(sdc_ipo_2, sdc_ipo_fin_variable, how='left', on='jkl_index')
sdc_ipo_merge_1

,jkl_index,deal_number_x,filing_date,issue_date,issuer,main_sic_code,nation,lead_underwriter_code,all_managers,all_mgr_role_code,...,gross_spread_as_%_of_prncpl_amt_in_this_mkt,low_price_of_filing_price_rnge,high_price_of_filing_price_rnge,change_in_value_of_shares_this_market_between_offer_and_1_day_after_offer_(us$_mil),market_value/_market_capitalization,market_value_after_offer_from_prospectus_($_mil),market_value_pro_forma_offer_($_mil),subscription_level,over_subscription_flag,subscription_shares
0,0,839870009.0,10/07/94,01/05/95,Genetronics Biomedical Corp,2836,UnitedStates,CANACCORD,Canaccord Capital Corp,BM,...,7.504,NaN,NaN,NaN,"103,640",NaN,NaN,NaN,NaN,NaN
1,5,452914002.0,09/19/94,01/10/95,DiaSys Corp,3841,UnitedStates,TASIN,Tasin & Company,BM,...,10.000,5.00,5.00,-5.000,NaN,NaN,10.0,NaN,NaN,NaN
2,9,444583002.0,10/20/94,01/10/95,Hauppauge Digital Inc,3577,UnitedStates,LIEBERBAUM,Lew Lieberbaum & Co.,BM,...,10.000,3.15,3.15,-4.200,100,NaN,NaN,NaN,NaN,NaN
3,11,444417002.0,11/08/94,01/11/95,MRV Communications Inc,3674,UnitedStates,HAMPSHIRE,Hampshire Securities Corp,BM,...,7.500,NaN,NaN,-12.000,"67,950",NaN,71.6,NaN,NaN,NaN
4,16,444885002.0,05/10/94,01/12/95,Developers Diversified Realty,6798,UnitedStates,DWR,Dean Witter Reynolds Inc,BM,...,5.31,NaN,NaN,-70.625,"2,643,130",NaN,456.1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33774,226352,4124156002.0,11/17/20,09/19/23,Klaviyo Inc,7372,UnitedStates,GS/MS-LLC(JB),Goldman Sachs & Co,JB,...,5.594,NaN,NaN,71.424,"647,420","7,555.7","7,555.7",NaN,NaN,NaN
33775,226364,4051569002.0,03/13/23,09/19/23,SR Bancorp Inc,6021,UnitedStates,KEEFE,Keefe Bruyette & Woods Inc,BM,...,na,NaN,NaN,NaN,"86,240",NaN,162.0,MIN,NaN,8500000.0
33776,226366,4134863002.0,09/19/23,09/19/23,Permian Resources Corp,1311,UnitedStates,JPM-SEC-LLC,JP Morgan Securities LLC,JB,...,2.750,NaN,NaN,5.148,"4,271,130","7,385.7","7,385.7",NaN,NaN,NaN
33777,226382,4133372002.0,09/18/23,09/20/23,Madison Sq Garden Ent Corp,7999,UnitedStates,BOFA-SEC-INC/G,BofA Securities Inc,JB,...,4.000,NaN,NaN,2.503,"1,451,450","1,387.9","1,387.9",NaN,NaN,NaN


# Exclude Extra Observation

### Exclude Non-IPO Offerings

In [ ]:
# I forgot to get the ipo_flag
# but its ok because we are going to match with preqin

### Keep only Offerings of Common Shares


In [ ]:
sdc_ipo_merge_1["type_of_security"].value_counts().head(15)

Common Shares      23288
Units               3358
Ord/Common Shs.     2314
Class A Shares      1745
ADS                 1291
Ordinary Shares      643
Ltd Prtnr Int        309
Shs Benficl Int      235
Class A Ord Shs      104
ADR                   82
Class B Shares        82
Stock Unit            52
Beneficial Ints       39
Sub Voting Shs        27
Trust Units           26
Name: type_of_security, dtype: int64

In [ ]:
sdc_ipo_merge_2 =sdc_ipo_merge_1[(sdc_ipo_merge_1['type_of_security'] =='Common Shares')|
                                 (sdc_ipo_merge_1['type_of_security'] =='Ord/Common Shs.')|
                                 (sdc_ipo_merge_1['type_of_security'] =='Class A Shares')|
                                 (sdc_ipo_merge_1['type_of_security'] =='Ordinary Shares')|
                                 (sdc_ipo_merge_1['type_of_security'] =='Class B Shares')|
                                 (sdc_ipo_merge_1['type_of_security'] =='Class A Ord Shs')]
sdc_ipo_merge_2

,jkl_index,deal_number_x,filing_date,issue_date,issuer,main_sic_code,nation,lead_underwriter_code,all_managers,all_mgr_role_code,...,gross_spread_as_%_of_prncpl_amt_in_this_mkt,low_price_of_filing_price_rnge,high_price_of_filing_price_rnge,change_in_value_of_shares_this_market_between_offer_and_1_day_after_offer_(us$_mil),market_value/_market_capitalization,market_value_after_offer_from_prospectus_($_mil),market_value_pro_forma_offer_($_mil),subscription_level,over_subscription_flag,subscription_shares
0,0,8.398700e+08,10/07/94,01/05/95,Genetronics Biomedical Corp,2836,UnitedStates,CANACCORD,Canaccord Capital Corp,BM,...,7.504,NaN,NaN,NaN,"103,640",NaN,NaN,NaN,NaN,NaN
1,5,4.529140e+08,09/19/94,01/10/95,DiaSys Corp,3841,UnitedStates,TASIN,Tasin & Company,BM,...,10.000,5.00,5.00,-5.000,NaN,NaN,10.0,NaN,NaN,NaN
2,9,4.445830e+08,10/20/94,01/10/95,Hauppauge Digital Inc,3577,UnitedStates,LIEBERBAUM,Lew Lieberbaum & Co.,BM,...,10.000,3.15,3.15,-4.200,100,NaN,NaN,NaN,NaN,NaN
3,11,4.444170e+08,11/08/94,01/11/95,MRV Communications Inc,3674,UnitedStates,HAMPSHIRE,Hampshire Securities Corp,BM,...,7.500,NaN,NaN,-12.000,"67,950",NaN,71.6,NaN,NaN,NaN
4,16,4.448850e+08,05/10/94,01/12/95,Developers Diversified Realty,6798,UnitedStates,DWR,Dean Witter Reynolds Inc,BM,...,5.31,NaN,NaN,-70.625,"2,643,130",NaN,456.1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33773,226348,4.134586e+09,09/19/23,09/19/23,HilleVax Inc,8731,UnitedStates,JPM-SEC-LLC,JP Morgan Securities LLC,JB,...,6.000,NaN,NaN,8.720,"631,360",590.2,590.2,NaN,NaN,NaN
33774,226352,4.124156e+09,11/17/20,09/19/23,Klaviyo Inc,7372,UnitedStates,GS/MS-LLC(JB),Goldman Sachs & Co,JB,...,5.594,NaN,NaN,71.424,"647,420","7,555.7","7,555.7",NaN,NaN,NaN
33775,226364,4.051569e+09,03/13/23,09/19/23,SR Bancorp Inc,6021,UnitedStates,KEEFE,Keefe Bruyette & Woods Inc,BM,...,na,NaN,NaN,NaN,"86,240",NaN,162.0,MIN,NaN,8500000.0
33776,226366,4.134863e+09,09/19/23,09/19/23,Permian Resources Corp,1311,UnitedStates,JPM-SEC-LLC,JP Morgan Securities LLC,JB,...,2.750,NaN,NaN,5.148,"4,271,130","7,385.7","7,385.7",NaN,NaN,NaN


### Exclude REITs


In [ ]:
sdc_ipo_merge_2['reit_type_code'].value_counts()

UN    1242
EQ     841
MO     247
HY      50
Name: reit_type_code, dtype: int64

In [ ]:
sdc_ipo_merge_3 =  sdc_ipo_merge_2[sdc_ipo_merge_2['reit_type_code'].isnull()]
sdc_ipo_merge_3.shape

(25796, 51)

### Exclude ADRs

In [ ]:
sdc_ipo_merge_3['depos_itary'].value_counts()

No    25604
Name: depos_itary, dtype: int64

In [ ]:
sdc_ipo_merge_4 =  sdc_ipo_merge_3[sdc_ipo_merge_3['depos_itary'] == 'No']
sdc_ipo_merge_4.shape

(25604, 51)

Exclude Closed-End Funds

In [ ]:
print(sdc_ipo_merge_4['closed_end_fund/trust_flag_(y/n)'].isnull().sum())
sdc_ipo_merge_4['closed_end_fund/trust_flag_(y/n)'].value_counts()

0


No     24818
Yes      786
Name: closed_end_fund/trust_flag_(y/n), dtype: int64

In [ ]:
sdc_ipo_merge_5 = sdc_ipo_merge_4[sdc_ipo_merge_4['closed_end_fund/trust_flag_(y/n)'] =='No']
sdc_ipo_merge_5.shape

(24818, 51)

### Exclude Units

In [ ]:
print(sdc_ipo_merge_5['unit_inv._trust'].isnull().sum())
sdc_ipo_merge_5['unit_inv._trust'].value_counts()

3772


No     20898
Yes      148
Name: unit_inv._trust, dtype: int64

In [ ]:
sdc_ipo_merge_6 = sdc_ipo_merge_5[sdc_ipo_merge_5['unit_inv._trust'] !='Yes']
sdc_ipo_merge_6.shape

(24670, 51)

### Exclude Penny Stocks
I am skeptical of excluding penny stocks.
I will just write the code for future usage.

In [ ]:
sdc_ipo_merge_6[sdc_ipo_merge_6['offer_price'] =='na']

11371    na
Name: offer_price, dtype: object

In [ ]:
# change the data to float
offer_price_list = []
for price in [price.replace(",", "") for price in sdc_ipo_merge_6['offer_price']]:
  try:
    tempt_1 = float(price)
    offer_price_list.append(tempt_1)
  except:
    tempt_2 = None
    offer_price_list.append(tempt_2)

# replace to the float data
sdc_ipo_merge_6['offer_price'] = offer_price_list

<ipython-input-87-dbfef32357a3>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdc_ipo_merge_6['offer_price'] = offer_price_list


In [ ]:
# just show the result. I wont use this criteria now
#  sdc_ipo_merge_6[sdc_ipo_merge_6['offer_price'] >= 5]

In [ ]:
# changed df names
sdc_ipo_merge_7 = sdc_ipo_merge_1
# change column names
sdc_ipo_merge_7.rename(columns={'deal_number_x': 'deal_number', 'cusip_x': 'cusip','9_digit_cusip_x':'cusip9'}, inplace=True)

### save the sdc preproccessing results

In [ ]:
# change the filing_date to other format
date_list = []
for date_f in sdc_ipo_merge_6['filing_date']:
  try:
    date_t1 = str(datetime.strptime(date_f, '%m/%d/%y').date())
    date_list.append(date_t1)
  except:
    date_t2 = None
    date_list.append(date_t2)
# replace to the float data
sdc_ipo_merge_6['filing_date'] = date_list

In [ ]:
# change the issue_date to other format
date_list_2 = []
for date_f_ in sdc_ipo_merge_6['issue_date']:
  try:
    date_t11 = str(datetime.strptime(date_f_, '%m/%d/%y').date())
    date_list_2.append(date_t11)
  except:
    date_t22 = None
    date_list_2.append(date_t22)
# replace to the float data
sdc_ipo_merge_6['issue_date'] = date_list_2

In [ ]:
# add variable issue year
sdc_ipo_merge_6['issue_year'] = [date__[:4] for date__ in sdc_ipo_merge_6['issue_date']]

In [ ]:
sdc_ipo_merge_6

,jkl_index,deal_number_x,filing_date,issue_date,issuer,main_sic_code,nation,lead_underwriter_code,all_managers,all_mgr_role_code,...,low_price_of_filing_price_rnge,high_price_of_filing_price_rnge,change_in_value_of_shares_this_market_between_offer_and_1_day_after_offer_(us$_mil),market_value/_market_capitalization,market_value_after_offer_from_prospectus_($_mil),market_value_pro_forma_offer_($_mil),subscription_level,over_subscription_flag,subscription_shares,issue_year
0,0,8.398700e+08,1994-10-07,1995-01-05,Genetronics Biomedical Corp,2836,UnitedStates,CANACCORD,Canaccord Capital Corp,BM,...,NaN,NaN,NaN,"103,640",NaN,NaN,NaN,NaN,NaN,1995
1,5,4.529140e+08,1994-09-19,1995-01-10,DiaSys Corp,3841,UnitedStates,TASIN,Tasin & Company,BM,...,5.00,5.00,-5.000,NaN,NaN,10.0,NaN,NaN,NaN,1995
2,9,4.445830e+08,1994-10-20,1995-01-10,Hauppauge Digital Inc,3577,UnitedStates,LIEBERBAUM,Lew Lieberbaum & Co.,BM,...,3.15,3.15,-4.200,100,NaN,NaN,NaN,NaN,NaN,1995
3,11,4.444170e+08,1994-11-08,1995-01-11,MRV Communications Inc,3674,UnitedStates,HAMPSHIRE,Hampshire Securities Corp,BM,...,NaN,NaN,-12.000,"67,950",NaN,71.6,NaN,NaN,NaN,1995
4,21,4.464180e+08,1994-10-20,1995-01-12,SunPharm Corp,2834,UnitedStates,ROYCE-INVEST,Royce Investment Group Inc,BM,...,7.00,7.00,-7.700,NaN,NaN,19.0,NaN,NaN,NaN,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24665,226348,4.134586e+09,2023-09-19,2023-09-19,HilleVax Inc,8731,UnitedStates,JPM-SEC-LLC,JP Morgan Securities LLC,JB,...,NaN,NaN,8.720,"631,360",590.2,590.2,NaN,NaN,NaN,2023
24666,226352,4.124156e+09,2020-11-17,2023-09-19,Klaviyo Inc,7372,UnitedStates,GS/MS-LLC(JB),Goldman Sachs & Co,JB,...,NaN,NaN,71.424,"647,420","7,555.7","7,555.7",NaN,NaN,NaN,2023
24667,226364,4.051569e+09,2023-03-13,2023-09-19,SR Bancorp Inc,6021,UnitedStates,KEEFE,Keefe Bruyette & Woods Inc,BM,...,NaN,NaN,NaN,"86,240",NaN,162.0,MIN,NaN,8500000.0,2023
24668,226366,4.134863e+09,2023-09-19,2023-09-19,Permian Resources Corp,1311,UnitedStates,JPM-SEC-LLC,JP Morgan Securities LLC,JB,...,NaN,NaN,5.148,"4,271,130","7,385.7","7,385.7",NaN,NaN,NaN,2023


In [ ]:
# cusip 8 column generate
sdc_ipo_merge_6['cusip8'] =''

In [ ]:
math.isnan(sdc_ipo_merge_6['9_digit_cusip_x'][-3:-2].iloc[0])

True

In [ ]:
sdc_ipo_merge_6['9_digit_cusip_x'].iloc[24667]

nan

In [ ]:
np.isfinite(sdc_ipo_merge_6['9_digit_cusip_x'].iloc[24668])

In [ ]:
sdc_ipo_merge_6_notnull = sdc_ipo_merge_6[sdc_ipo_merge_6['9_digit_cusip_x'].notnull()]
sdc_ipo_merge_6_notnull['cusip8'] = sdc_ipo_merge_6_notnull['9_digit_cusip_x'].astype(str).str[:8]

<ipython-input-150-85db5138caf1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdc_ipo_merge_6_notnull['cusip8'] = sdc_ipo_merge_6_notnull['9_digit_cusip_x'].astype(str).str[:8]


In [ ]:
sdc_ipo_merge_6_null = sdc_ipo_merge_6[sdc_ipo_merge_6['9_digit_cusip_x'].isnull()]
sdc_ipo_merge_6_null['cusip8'] = sdc_ipo_merge_6_null['cusip_x'].iloc[i] + '10'

<ipython-input-153-133c7cfc9905>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdc_ipo_merge_6_null['cusip8'] = sdc_ipo_merge_6_null['cusip_x'].iloc[i] + '10'


In [ ]:
sdc_ipo_merge_7 = pd.concat([sdc_ipo_merge_6_notnull,sdc_ipo_merge_6_null])
sdc_ipo_merge_7.sort_values(by=["jkl_index"],inplace=True)
sdc_ipo_merge_7

,jkl_index,deal_number_x,filing_date,issue_date,issuer,main_sic_code,nation,lead_underwriter_code,all_managers,all_mgr_role_code,...,high_price_of_filing_price_rnge,change_in_value_of_shares_this_market_between_offer_and_1_day_after_offer_(us$_mil),market_value/_market_capitalization,market_value_after_offer_from_prospectus_($_mil),market_value_pro_forma_offer_($_mil),subscription_level,over_subscription_flag,subscription_shares,issue_year,cusip8
0,0,8.398700e+08,1994-10-07,1995-01-05,Genetronics Biomedical Corp,2836,UnitedStates,CANACCORD,Canaccord Capital Corp,BM,...,NaN,NaN,"103,640",NaN,NaN,NaN,NaN,NaN,1995,37191610
1,5,4.529140e+08,1994-09-19,1995-01-10,DiaSys Corp,3841,UnitedStates,TASIN,Tasin & Company,BM,...,5.00,-5.000,NaN,NaN,10.0,NaN,NaN,NaN,1995,25283810
2,9,4.445830e+08,1994-10-20,1995-01-10,Hauppauge Digital Inc,3577,UnitedStates,LIEBERBAUM,Lew Lieberbaum & Co.,BM,...,3.15,-4.200,100,NaN,NaN,NaN,NaN,NaN,1995,41913120
3,11,4.444170e+08,1994-11-08,1995-01-11,MRV Communications Inc,3674,UnitedStates,HAMPSHIRE,Hampshire Securities Corp,BM,...,NaN,-12.000,"67,950",NaN,71.6,NaN,NaN,NaN,1995,55347710
4,21,4.464180e+08,1994-10-20,1995-01-12,SunPharm Corp,2834,UnitedStates,ROYCE-INVEST,Royce Investment Group Inc,BM,...,7.00,-7.700,NaN,NaN,19.0,NaN,NaN,NaN,1995,86764820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24665,226348,4.134586e+09,2023-09-19,2023-09-19,HilleVax Inc,8731,UnitedStates,JPM-SEC-LLC,JP Morgan Securities LLC,JB,...,NaN,8.720,"631,360",590.2,590.2,NaN,NaN,NaN,2023,43157M10
24666,226352,4.124156e+09,2020-11-17,2023-09-19,Klaviyo Inc,7372,UnitedStates,GS/MS-LLC(JB),Goldman Sachs & Co,JB,...,NaN,71.424,"647,420","7,555.7","7,555.7",NaN,NaN,NaN,2023,49845K10
24667,226364,4.051569e+09,2023-03-13,2023-09-19,SR Bancorp Inc,6021,UnitedStates,KEEFE,Keefe Bruyette & Woods Inc,BM,...,NaN,NaN,"86,240",NaN,162.0,MIN,NaN,8500000.0,2023,02957910
24668,226366,4.134863e+09,2023-09-19,2023-09-19,Permian Resources Corp,1311,UnitedStates,JPM-SEC-LLC,JP Morgan Securities LLC,JB,...,NaN,5.148,"4,271,130","7,385.7","7,385.7",NaN,NaN,NaN,2023,71424F10


# Matching SDC and CRSP database

### use the whole data 1995 to 2022

In [ ]:
# get CRSP daily stock fillings
crsp_1995_2022 =  pd.read_csv('https://utexas.box.com/shared/static/unuysf43vpet2b1bpw8uu0m1bv54gguw.csv')
crsp_1995_2022

<ipython-input-2-751a83fc2337>:1: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp_1995_2022 =  pd.read_csv('https://utexas.box.com/shared/static/unuysf43vpet2b1bpw8uu0m1bv54gguw.csv')


,PERMNO,date,SHRCD,EXCHCD,NCUSIP,PRC,RET,SHROUT,OPENPRC
0,10001,1995-01-03,11,3,29274A10,8.375,0.046875,2224,8.500
1,10001,1995-01-04,11,3,29274A10,8.000,-0.044776,2224,8.000
2,10001,1995-01-05,11,3,29274A10,7.500,-0.062500,2224,8.000
3,10001,1995-01-06,11,3,29274A10,-7.750,0.033333,2224,NaN
4,10001,1995-01-09,11,3,29274A10,7.500,-0.032258,2224,7.500
...,...,...,...,...,...,...,...,...,...
40917855,93436,2022-12-23,11,3,88160R10,123.150,-0.017551,3157752,126.370
40917856,93436,2022-12-27,11,3,88160R10,109.100,-0.114089,3157752,117.495
40917857,93436,2022-12-28,11,3,88160R10,112.710,0.033089,3157752,110.350
40917858,93436,2022-12-29,11,3,88160R10,121.820,0.080827,3157752,120.385


In [ ]:
# keep only common shares
crsp_1995_2022 = crsp_1995_2022[(crsp_1995_2022['SHRCD']>= 10)&(crsp_1995_2022['SHRCD']<= 19)]
# NYSE, NASDAQ or AMEX
crsp_1995_2022 = crsp_1995_2022[(crsp_1995_2022['EXCHCD']== 1)|(crsp_1995_2022['EXCHCD']==2)|(crsp_1995_2022['EXCHCD']==3)].copy()
# exclude rows where price information is missing
crsp_1995_2022 = crsp_1995_2022[crsp_1995_2022['PRC'].notnull()]
crsp_1995_2022.shape

(40917860, 9)

In [ ]:
# sorted by PERMNO and date
crsp_1995_2022.sort_values(by=['PERMNO','date'],inplace=True)

In [ ]:
# create NCUSIP6 variable, equal to the first six characters of the NCUSIP variable
crsp_1995_2022['NCUSIP6'] =  crsp_1995_2022['NCUSIP'].astype(str).str[:6]

# From CUSIP to PERMNO

#### cusip to permno using 8 digit cusip

In [ ]:
# this is the match function equivalent with match in R code
def match_fast_2(a: List[Hashable], b: List[Hashable]) -> List[int]:
  test_dic ={}
  for i, x in enumerate(tqdm(b)):
    if x in test_dic:
      pass
    else:
      test_dic.update({x:i})
  return [test_dic.get(x, None) for x in a]

In [ ]:
# match the CUSIP8 variable in the ipo data to the NCUSIP variable from crsp
match_list_1 = match_fast_2(list(sdc_ipo_merge_7['cusip8']),list(crsp_1995_2022['NCUSIP']))
match_list_1

In [ ]:
# put the indexs in the df
sdc_ipo_merge_7['match_list_1'] = match_list_1

# get the permno from the crsp using indexs from above
Permno_ncusip_list = []
for indx in sdc_ipo_merge_7['match_list_1']:
  try:
    tempt_1 = int(indx)
    Permno_ncusip_list.append(crsp_1995_2022['PERMNO'].iloc[tempt_1])
  except:
    tempt_2 = None
    Permno_ncusip_list.append(tempt_2)

In [ ]:
# save the permno
sdc_ipo_merge_7['Permno_ncusip'] = Permno_ncusip_list
sdc_ipo_merge_7.shape

(24670, 52)

In [ ]:
# get the first date from cusip8

In [ ]:
# get the date index from matching permnos
match_date_1 = match_fast_2(list(sdc_ipo_merge_7['Permno_ncusip']),list(crsp_1995_2022['PERMNO']))
# match date in df
sdc_ipo_merge_7['match_date_1'] = match_date_1

100%|██████████| 40917860/40917860 [00:11<00:00, 3704049.81it/s]


In [ ]:
# get the first date of crsp using indexs
first_date_list1 = []
for indx in sdc_ipo_merge_7['match_date_1']:
  try:
    tempt_1 = int(indx)
    first_date_list1.append(crsp_1995_2022['date'].iloc[tempt_1])
  except:
    tempt_2 = None
    first_date_list1.append(tempt_2)

In [ ]:
# save the first date
sdc_ipo_merge_7['First_CRSP_date_ncusip'] = first_date_list1
sdc_ipo_merge_7.shape

(24670, 54)

In [ ]:
# looks right
# sdc_ipo_merge_7.loc[:,['filing_date','issue_date','First_CRSP_date_ncusip']].iloc[9000:9040]

#### another cusip to permno using 6 digit cusip

In [ ]:
# we have to use the first 6 digits from cusip8 not just the cusip from sdc
# diff_list =  np.where(sdc_ipo_merge_7['cusip'] != sdc_ipo_merge_7['cusip8'].str[:6])
# sdc_ipo_merge_7[['cusip','cusip8']].iloc[diff_list[0]]

In [ ]:
# match the CUSIP8 variable in the ipo data to the NCUSIP variable from crsp
match_list_2 = match_fast_2(list(sdc_ipo_merge_7['cusip8'].str[:6]),list(crsp_1995_2022['NCUSIP6']))
match_list_2

In [ ]:
# put the indexs in the df
sdc_ipo_merge_7['match_list_2'] = match_list_2

# get the permno from the crsp using indexs from above
Permno_ncusip6_list = []
for indx in sdc_ipo_merge_7['match_list_2']:
  try:
    tempt_1 = int(indx)
    Permno_ncusip6_list.append(crsp_1995_2022['PERMNO'].iloc[tempt_1])
  except:
    tempt_2 = None
    Permno_ncusip6_list.append(tempt_2)

In [ ]:
# save the permno
sdc_ipo_merge_7['Permno_ncusip6'] = Permno_ncusip6_list
sdc_ipo_merge_7.shape

(24670, 56)

In [ ]:
# get the first date from cusip6

In [ ]:
# get the date index from matching permnos
match_date_2 = match_fast_2(list(sdc_ipo_merge_7['Permno_ncusip6']),list(crsp_1995_2022['PERMNO']))
# match date in df
sdc_ipo_merge_7['match_date_2'] = match_date_2

100%|██████████| 40917860/40917860 [00:11<00:00, 3703340.29it/s]


In [ ]:
# get the first date of crsp using indexs
first_date_list2 = []
for indx in sdc_ipo_merge_7['match_date_2']:
  try:
    tempt_1 = int(indx)
    first_date_list2.append(crsp_1995_2022['date'].iloc[tempt_1])
  except:
    tempt_2 = None
    first_date_list2.append(tempt_2)

In [ ]:
# save the first date
sdc_ipo_merge_7['First_CRSP_date_ncusip6'] = first_date_list2
sdc_ipo_merge_7.shape

(24670, 58)

In [ ]:
# looks right
sdc_ipo_merge_7.loc[:,['filing_date','issue_date','First_CRSP_date_ncusip','First_CRSP_date_ncusip6']].iloc[23000:23040]

### Observations with CRSP Record in + - 7 days

We match the ipo data to the crsp data in two ways, by both CUSIP8 and by six-digit cusip. Specifically, we first
match the CUSIP8 variable in the ipo data to the NCUSIP variable from crsp. We use crsp data to find the Permno
and the date when the security first appears on CRSP. These variables are called Permno_ncusip and
First_CRSP_date_ncusip.

In [ ]:
# change to datetime type
sdc_ipo_merge_8['First_CRSP_date_ncusip'] = pd.to_datetime(sdc_ipo_merge_8['First_CRSP_date_ncusip'])
sdc_ipo_merge_8['First_CRSP_date_ncusip6'] = pd.to_datetime(sdc_ipo_merge_8['First_CRSP_date_ncusip6'])
sdc_ipo_merge_8['issue_date'] = pd.to_datetime(sdc_ipo_merge_8['issue_date'])

In [ ]:
sdc_ipo_merge_8['date_diff_1'] = sdc_ipo_merge_8['First_CRSP_date_ncusip'] - sdc_ipo_merge_8['issue_date']
sdc_ipo_merge_8['date_diff_1'] = [day.days for day in sdc_ipo_merge_8['date_diff_1']]
sdc_ipo_merge_8

,jkl_index,deal_number,filing_date,issue_date,issuer,main_sic_code,nation,lead_underwriter_code,all_managers,all_mgr_role_code,...,cusip8,match_list_1,Permno_ncusip,match_date_1,First_CRSP_date_ncusip,match_list_2,Permno_ncusip6,match_date_2,First_CRSP_date_ncusip6,date_diff_1
0,0,8.398700e+08,1994-10-07,1995-01-05,Genetronics Biomedical Corp,2836,UnitedStates,CANACCORD,Canaccord Capital Corp,BM,...,37191610,31930372.0,86444.0,31930372.0,1998-12-08,31930372.0,86444.0,31930372.0,1998-12-08,1433.0
1,5,4.529140e+08,1994-09-19,1995-01-10,DiaSys Corp,3841,UnitedStates,TASIN,Tasin & Company,BM,...,25283810,24537437.0,81201.0,24537437.0,1995-01-10,24537437.0,81201.0,24537437.0,1995-01-10,0.0
2,9,4.445830e+08,1994-10-20,1995-01-10,Hauppauge Digital Inc,3577,UnitedStates,LIEBERBAUM,Lew Lieberbaum & Co.,BM,...,41913120,NaN,NaN,NaN,NaT,24923299.0,81562.0,24923299.0,1995-04-25,NaN
3,11,4.444170e+08,1994-11-08,1995-01-11,MRV Communications Inc,3674,UnitedStates,HAMPSHIRE,Hampshire Securities Corp,BM,...,55347710,19788099.0,78179.0,19788099.0,1995-01-03,19788099.0,78179.0,19788099.0,1995-01-03,-8.0
4,21,4.464180e+08,1994-10-20,1995-01-12,SunPharm Corp,2834,UnitedStates,ROYCE-INVEST,Royce Investment Group Inc,BM,...,86764820,NaN,NaN,NaN,NaT,24600235.0,81228.0,24600235.0,1995-01-12,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24665,226348,4.134586e+09,2023-09-19,2023-09-19,HilleVax Inc,8731,UnitedStates,JPM-SEC-LLC,JP Morgan Securities LLC,JB,...,43157M10,8010854.0,23032.0,8010854.0,2022-04-29,8010854.0,23032.0,8010854.0,2022-04-29,-508.0
24666,226352,4.124156e+09,2020-11-17,2023-09-19,Klaviyo Inc,7372,UnitedStates,GS/MS-LLC(JB),Goldman Sachs & Co,JB,...,49845K10,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN
24667,226364,4.051569e+09,2023-03-13,2023-09-19,SR Bancorp Inc,6021,UnitedStates,KEEFE,Keefe Bruyette & Woods Inc,BM,...,02957910,23807595.0,80766.0,23807595.0,1995-01-03,23807595.0,80766.0,23807595.0,1995-01-03,-10486.0
24668,226366,4.134863e+09,2023-09-19,2023-09-19,Permian Resources Corp,1311,UnitedStates,JPM-SEC-LLC,JP Morgan Securities LLC,JB,...,71424F10,5282778.0,16021.0,5281175.0,2016-04-15,5282778.0,16021.0,5281175.0,2016-04-15,-2713.0


In [ ]:
sdc_ipo_date_1 = sdc_ipo_merge_8[((sdc_ipo_merge_8['date_diff_1'] >=-3)&(sdc_ipo_merge_8['date_diff_1'] <=10))|(sdc_ipo_merge_8['date_diff_1'].isnull())].copy()
sdc_ipo_date_1

In [ ]:
sdc_ipo_date_1['date_diff_2'] = sdc_ipo_date_1['First_CRSP_date_ncusip6'] - sdc_ipo_date_1['issue_date']
sdc_ipo_date_1['date_diff_2'] = [day.days for day in sdc_ipo_date_1['date_diff_2']]
sdc_ipo_date_1

In [ ]:
# date diff bigger than -5 smaller than 10
date_diff_3= list(sdc_ipo_date_1['date_diff_1'].copy())
for i, ddiff in enumerate(sdc_ipo_date_1['date_diff_1']):
  if np.isnan(ddiff):
    date_diff_3[i] = sdc_ipo_date_1['date_diff_2'].iloc[i]
sdc_ipo_date_1['date_diff_3'] =date_diff_3

In [ ]:
sdc_ipo_date_1 = sdc_ipo_date_1[(sdc_ipo_date_1['date_diff_3'] >-5)&(sdc_ipo_date_1['date_diff_3'] <10)]
sdc_ipo_date_1.drop(["date_diff_1","date_diff_2"],axis=1, inplace=True)
sdc_ipo_date_1

<ipython-input-47-941aae9fa583>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdc_ipo_date_1.drop(["date_diff_1","date_diff_2"],axis=1, inplace=True)


,jkl_index,deal_number,filing_date,issue_date,issuer,main_sic_code,nation,lead_underwriter_code,all_managers,all_mgr_role_code,...,cusip8,match_list_1,Permno_ncusip,match_date_1,First_CRSP_date_ncusip,match_list_2,Permno_ncusip6,match_date_2,First_CRSP_date_ncusip6,date_diff_3
1,5,4.529140e+08,1994-09-19,1995-01-10,DiaSys Corp,3841,UnitedStates,TASIN,Tasin & Company,BM,...,25283810,24537437.0,81201.0,24537437.0,1995-01-10,24537437.0,81201.0,24537437.0,1995-01-10,0.0
4,21,4.464180e+08,1994-10-20,1995-01-12,SunPharm Corp,2834,UnitedStates,ROYCE-INVEST,Royce Investment Group Inc,BM,...,86764820,NaN,NaN,NaN,NaT,24600235.0,81228.0,24600235.0,1995-01-12,0.0
6,29,4.455930e+08,1994-10-13,1995-01-18,California Pro Sports Inc,5091,UnitedStates,COHIG,Cohig & Associates,BM,...,13055110,24533380.0,81199.0,24533380.0,1995-01-18,24533380.0,81199.0,24533380.0,1995-01-18,0.0
7,30,4.460520e+08,1994-09-14,1995-01-18,Infosafe Systems Inc,7372,UnitedStates,D-H-BLAIR,D. H. Blair,BM,...,45691320,NaN,NaN,NaN,NaT,24561016.0,81212.0,24561016.0,1995-01-18,0.0
8,32,4.463820e+08,1994-10-18,1995-01-18,Timeline Inc,7372,UnitedStates,T-JAMES,Thomas James Associates,BM,...,88733620,NaN,NaN,NaN,NaT,24602372.0,81230.0,24602372.0,1995-01-19,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23886,221800,3.989232e+09,2022-10-17,2022-11-14,Acrivon Therapeutics Inc,8731,UnitedStates,MS/JEFFERIES-L,Morgan Stanley & Co,JB,...,00489010,8109703.0,23510.0,8109703.0,2022-11-15,8109703.0,23510.0,8109703.0,2022-11-15,1.0
23936,222075,4.015683e+09,NaN,2022-12-06,Adamas One Corp,5094,UnitedStates,NOTAPP,Not Applicable,BM,...,00548H10,8135912.0,23622.0,8135912.0,2022-12-09,8135912.0,23622.0,8135912.0,2022-12-09,3.0
23937,222077,3.928598e+09,2022-05-31,2022-12-06,Adamas One Corp,5094,UnitedStates,ALEXANDER-LP,Alexander Capital LP,BM,...,00548H10,8135912.0,23622.0,8135912.0,2022-12-09,8135912.0,23622.0,8135912.0,2022-12-09,3.0
23966,222256,3.956107e+09,2022-07-29,2022-12-19,Lipella Pharmaceuticals Inc,2836,UnitedStates,SPARTAN-CAPITA,Spartan Capital Securities LL,BM,...,53630L10,8135944.0,23625.0,8135944.0,2022-12-20,8135944.0,23625.0,8135944.0,2022-12-20,1.0


In [ ]:
# date diff bigger than -5 smaller than 10
r_permno = sdc_ipo_date['Permno_ncusip'].copy()
for i, prno in enumerate(sdc_ipo_date['Permno_ncusip']):
  if np.isnan(prno):
    r_permno[i] = sdc_ipo_date['Permno_ncusip6'].iloc[i]
sdc_ipo_date['r_permno'] =r_permno
sdc_ipo_date

In [ ]:
sdc_ipo_date.drop(['match_list_1', 'Permno_ncusip','match_list_2', 'Permno_ncusip6'],axis=1, inplace=True)

# Variable Construction

### initial returns

In [ ]:
def match_fast_2(a: List[Hashable], b: List[Hashable]) -> List[int]:
  test_dic ={}
  for i, x in enumerate(tqdm(b)):
    if x in test_dic:
      pass
    else:
      test_dic.update({x:i})
  return [test_dic.get(x, None) for x in a]

In [ ]:
# match the r_permno variable in the ipo data to the permno variable from crsp
match_permno = match_fast_2(list(sdc_ipo_date['r_permno']),list(crsp_1995_2022['PERMNO']))
match_permno

In [ ]:
# close price from CRSP
sdc_ipo_date['Close_Price'] = list(crsp_1995_2022.iloc[match_permno,:]['PRC'])
#  CRSP data includes negative prices. To adjust for this caveat in the data we take the absolute
sdc_ipo_date['Close_Price'] = sdc_ipo_date['Close_Price'].apply(abs)

In [ ]:
# innitial retrun
sdc_ipo_date['initial_return'] = sdc_ipo_date['Close_Price']/sdc_ipo_date['offer_price'] - 1

In [ ]:
sdc_ipo_date.columns

Index(['jkl_index', 'deal_number', 'filing_date', 'issue_date', 'issuer',
       'main_sic_code', 'nation', 'lead_underwriter_code', 'all_managers',
       'all_mgr_role_code', 'lead_underwriter', 'cusip', 'cusip9',
       'lead_underwriter_list', 'JB_JL_jointlead', 'CM_CL_comanager',
       'gross_spread_($)', 'manage_ment_fee_($)', 'under_writing_fee_($)',
       'selling_conces_sion_($)', 're_allow_ance_fee_($)',
       'gross_spread_in_this_mkt_($_mil)', 'principal_amount_($_mil)',
       'proceeds_amt_in_this_mkt_($_mil)', 'offer_price', 'type_of_security',
       'description', 'marketplace',
       'primary_exchange_where_issue_will_be_listed',
       'original_high_filing_price', 'original_low_filing_price',
       'original_middle_of_filing_price_range', 'shares_filed_in_this_mkt',
       'amt_filed_in_this_mkt_($_mil)', 'reit_type_code', 'depos_itary',
       'unit_inv._trust', 'closed_end_fund/trust_flag_(y/n)',
       'gross_spread_as_%_of_prncpl_amt_in_this_mkt',
       'l

In [ ]:
# remove the commas in proceeds
sdc_ipo_date['proceeds_amt_in_this_mkt_($_mil)'] = [float(procc.replace(",", "")) for procc in sdc_ipo_date['proceeds_amt_in_this_mkt_($_mil)']]

### proceeds in real terms 2022

In [ ]:
# frad gdp data
gdp = pd.read_csv('/content/drive/MyDrive/UT_research/VC_IB/data/GDPDEF.csv')
# the first quarter date
gdp['year'] = [d[:4] for d in gdp['DATE']]
gdp = gdp.drop_duplicates(subset=['year'])

In [ ]:
gdp

,DATE,GDPDEF,year
0,1947-01-01,11.141,1947
4,1948-01-01,11.865,1948
8,1949-01-01,12.165,1949
12,1950-01-01,11.970,1950
16,1951-01-01,12.953,1951
...,...,...,...
288,2019-01-01,103.375,2019
292,2020-01-01,105.037,2020
296,2021-01-01,107.668,2021
300,2022-01-01,115.135,2022


In [ ]:
gdp['base'] = 115.135
gdp['factor_2022']=gdp['base']/gdp['GDPDEF']

In [ ]:
gdp

,DATE,GDPDEF,year,base,factor_2022
0,1947-01-01,11.141,1947,115.135,10.334351
4,1948-01-01,11.865,1948,115.135,9.703751
8,1949-01-01,12.165,1949,115.135,9.464447
12,1950-01-01,11.970,1950,115.135,9.618630
16,1951-01-01,12.953,1951,115.135,8.888674
...,...,...,...,...,...
288,2019-01-01,103.375,2019,115.135,1.113761
292,2020-01-01,105.037,2020,115.135,1.096138
296,2021-01-01,107.668,2021,115.135,1.069352
300,2022-01-01,115.135,2022,115.135,1.000000


In [ ]:
gdp_match = match_fast_2(sdc_ipo_date['issue_year'],gdp['year'].astype(int) )
gdp_match

In [ ]:
sdc_ipo_date['factor_2022'] =  list(gdp['factor_2022'].iloc[gdp_match])
sdc_ipo_date

In [ ]:
sdc_ipo_date['Proceeds_2022'] = sdc_ipo_date['factor_2022'] * sdc_ipo_date['proceeds_amt_in_this_mkt_($_mil)']

In [ ]:
sdc_ipo_date

,jkl_index,deal_number,filing_date,issue_date,issuer,main_sic_code,nation,lead_underwriter_code,all_managers,all_mgr_role_code,...,match_date_1,First_CRSP_date_ncusip,match_date_2,First_CRSP_date_ncusip6,date_diff_3,r_permno,Close_Price,initial_return,factor_2022,Proceeds_2022
0,5,4.529140e+08,1994-09-19,1995-01-10,DiaSys Corp,3841,UnitedStates,TASIN,Tasin & Company,BM,...,24537437.0,1995-01-10,24537437.0,1995-01-10,0.0,81201.0,5.6875,0.137500,1.732604,8.663020
1,21,4.464180e+08,1994-10-20,1995-01-12,SunPharm Corp,2834,UnitedStates,ROYCE-INVEST,Royce Investment Group Inc,BM,...,NaN,NaN,24600235.0,1995-01-12,0.0,81228.0,7.5000,0.071429,1.732604,13.341051
2,29,4.455930e+08,1994-10-13,1995-01-18,California Pro Sports Inc,5091,UnitedStates,COHIG,Cohig & Associates,BM,...,24533380.0,1995-01-18,24533380.0,1995-01-18,0.0,81199.0,4.5000,0.000000,1.732604,9.356062
3,30,4.460520e+08,1994-09-14,1995-01-18,Infosafe Systems Inc,7372,UnitedStates,D-H-BLAIR,D. H. Blair,BM,...,NaN,NaN,24561016.0,1995-01-18,0.0,81212.0,5.2500,0.050000,1.732604,13.514311
4,32,4.463820e+08,1994-10-18,1995-01-18,Timeline Inc,7372,UnitedStates,T-JAMES,Thomas James Associates,BM,...,NaN,NaN,24602372.0,1995-01-19,1.0,81230.0,4.2500,-0.150000,1.732604,8.663020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5994,221800,3.989232e+09,2022-10-17,2022-11-14,Acrivon Therapeutics Inc,8731,UnitedStates,MS/JEFFERIES-L,Morgan Stanley & Co,JB,...,8109703.0,2022-11-15,8109703.0,2022-11-15,1.0,23510.0,16.6400,0.331200,1.000000,94.400000
5995,222075,4.015683e+09,NaN,2022-12-06,Adamas One Corp,5094,UnitedStates,NOTAPP,Not Applicable,BM,...,8135912.0,2022-12-09,8135912.0,2022-12-09,3.0,23622.0,4.7400,0.053333,1.000000,20.100000
5996,222077,3.928598e+09,2022-05-31,2022-12-06,Adamas One Corp,5094,UnitedStates,ALEXANDER-LP,Alexander Capital LP,BM,...,8135912.0,2022-12-09,8135912.0,2022-12-09,3.0,23622.0,4.7400,0.053333,1.000000,11.000000
5997,222256,3.956107e+09,2022-07-29,2022-12-19,Lipella Pharmaceuticals Inc,2836,UnitedStates,SPARTAN-CAPITA,Spartan Capital Securities LL,BM,...,8135944.0,2022-12-20,8135944.0,2022-12-20,1.0,23625.0,4.7100,-0.180870,1.000000,16.800000


### initial price range

In [ ]:
#change columns
sdc_ipo_infla = sdc_ipo_date
sdc_ipo_infla.rename(columns={"original_high_filing_price": "HFILE", "original_low_filing_price": "LFILE",
                              "high_price_of_filing_price_rnge": "AH_HFILE", "low_price_of_filing_price_rnge": "AH_LFILE"}, inplace=True)

In [ ]:
# Estimation of Initial Price Range
def numer_func(x):
    return pd.to_numeric(x.replace(",", ""), errors='coerce')

# Add a new column 'Issue_priced_range' initialized with "NA"
sdc_ipo_infla['Issue_priced_range'] = "NA"

In [ ]:
sdc_ipo_infla['constructed_low'] = numer_func(sdc_ipo_infla['LFILE'])
sdc_ipo_infla['constructed_high'] = numer_func(sdc_ipo_infla['HFILE'])

In [ ]:
# constructed_low
constructed_low= list(sdc_ipo_infla['constructed_low'].copy())
for i, pprice in enumerate(sdc_ipo_infla['constructed_low'].copy()):
  if np.isnan(pprice):
    constructed_low[i] = sdc_ipo_infla['AH_LFILE'].iloc[i]
sdc_ipo_infla['constructed_low'] = constructed_low

# constructed_high
constructed_high= list(sdc_ipo_infla['constructed_high'].copy())
for i, pprice in enumerate(sdc_ipo_infla['constructed_high'].copy()):
  if np.isnan(pprice):
    constructed_high[i] = sdc_ipo_infla['AH_HFILE'].iloc[i]
sdc_ipo_infla['constructed_high'] = constructed_high

In [ ]:
# Sometimes SDC reverses the low and high ends of the price range. To adjust for such mistakes we create
# variables min_range which equals the minimum of the low and high ends, and max_range which equals the
# maximum.

# Calculate the 'min_range' column
sdc_ipo_infla['min_range'] = 0.5 * (sdc_ipo_infla['constructed_low'] + sdc_ipo_infla['constructed_high']) - 0.5 * abs(sdc_ipo_infla['constructed_low'] - sdc_ipo_infla['constructed_high'])

# Calculate the 'max_range' column
sdc_ipo_infla['max_range'] = 0.5 * (sdc_ipo_infla['constructed_low'] + sdc_ipo_infla['constructed_high']) + 0.5 * abs(sdc_ipo_infla['constructed_low'] - sdc_ipo_infla['constructed_high'])

In [ ]:
# Create a boolean condition for not missing values in 'constructed_low' and 'constructed_high'
not_missing = (~sdc_ipo_infla['constructed_low'].isna()) & (~sdc_ipo_infla['constructed_high'].isna())

# Create the 'Issue_priced_range' column based on conditions
sdc_ipo_infla.loc[not_missing & (sdc_ipo_infla['offer_price'] < sdc_ipo_infla['min_range']), 'Issue_priced_range'] = "Below range"
sdc_ipo_infla.loc[not_missing & (sdc_ipo_infla['offer_price'] >= sdc_ipo_infla['min_range']) & (sdc_ipo_infla['offer_price'] <= sdc_ipo_infla['max_range']), 'Issue_priced_range'] = "Within range"
sdc_ipo_infla.loc[not_missing & (sdc_ipo_infla['offer_price'] > sdc_ipo_infla['max_range']), 'Issue_priced_range'] = "Above range"

In [ ]:
sdc_ipo_infla['Issue_priced_range'].value_counts()

NA              2056
Within range    1978
Above range     1048
Below range      917
Name: Issue_priced_range, dtype: int64